[View in Colaboratory](https://colab.research.google.com/github/nkcr/WEM-labs/blob/master/Projet/cleanner/Pre_processing.ipynb)

Ce notebook contient les scripts pour le projet Per-Tempus

# Pré-traitement

Set the mongoDB configuration URL. DO NOT LEAVE IT HERE FOR GIT VERSIONING

In [0]:
MD="???"

## Dependencies and Mongo Client

In [2]:
!pip install pymongo

from pymongo import MongoClient
import numpy as np

client = MongoClient(MD)
db = client.news

    100% |████████████████████████████████| 389kB 6.4MB/s 


In [3]:
print(db.swissinfo.find().count())

15483


In [4]:
source = [
    "swissinfo",
    "rts",
    "leTemps",
    "20Minutes"
]

totals = list( map( lambda collection: db[collection].find().count(), source ) )
total = np.sum(totals)

print(total)

133228


## Generate RAW corpus

In [77]:
import nltk
from multiprocessing.pool import ThreadPool
from multiprocessing import Pool
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')


# The leTemps dataset has 159534 different tokens

corpus = set()

def clean_str(string):
    """Original taken
    from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9()èéàÉÀêÊçÇ]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()
  
def tokenize(text):
  return word_tokenize(text)

for article in db.allArticles.find({},{"clearedHTML":1}):
  [corpus.add(token) for token in tokenize(clean_str(article["clearedHTML"]))]
  
print(len(corpus))

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
414738


In [78]:
len(corpus)

414738

### Save RAW corpus

In [0]:
db_corpus = client.corpus

In [80]:
list_corpus = list(corpus)
db_corpus.allArticles.insert_one(
    {
        "id": "raw2",
        "corpus": list_corpus,
        "description": "uniq words tokenized by nltk on all articles. Words are\
          only processed with `clean_str` UPDATE WITH ACCENTS"
    }
)

### Get RAW corpus, remove small tokens and add it to Mongo

We estimate that the collection contains only one document

In [0]:
list_raw_corpus = db_corpus.allArticles.find_one({"id": "raw2"})["corpus"]

In [83]:
print(len(list_corpus))

larger = list(filter(lambda w: len(w) > 2, list_corpus))

print(len(larger))

414738
413630


In [84]:
#
# An example to add a field
#
#db_corpus.allArticles.find_one_and_update( {},{'$set':{"raw_more_than_two_chars": larger}} )

db_corpus.allArticles.insert_one(
    {
        "id": "greater_than_two2",
        "corpus": larger,
        "description": "Raw corpus with only tokens greater than 2 chars WITH UPDATED ACCENTS"
    }
)

### Remove numeric tokens and add it

In [86]:
print(len(larger))

condition = lambda w: not w.replace('"', '').replace("'", '')\
          .replace(".", '').replace(",", '').isnumeric()

alpha_only = list(filter(condition, larger))

print(len(alpha_only))

413630
407229


In [88]:
db_corpus.allArticles.insert_one(
    {
        "id": "only_alpha2",
        "corpus": alpha_only,
        "description": "`greater_than_two` without numeric tokens WITH UPDATED ACCENT"
    }
)

### Special chars and lower

In [0]:
from nltk.tokenize import word_tokenize

In [0]:

db_corpus = client.corpus
alpha_only = db_corpus.allArticles.find_one({"id": "only_alpha2"})["corpus"]

In [91]:
print(len(alpha_only))

def clean_str(string):
  """Original taken
  from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
  """
  string = re.sub(r"[^A-Za-z0-9()èéàÉÀêÊçÇ]", " ", string)
  string = re.sub(r"\'s", " \'s", string)
  string = re.sub(r"\'ve", " \'ve", string)
  string = re.sub(r"n\'t", " n\'t", string)
  string = re.sub(r"\'re", " \'re", string)
  string = re.sub(r"\'d", " \'d", string)
  string = re.sub(r"\'ll", " \'ll", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", " \( ", string)
  string = re.sub(r"\)", " \) ", string)
  string = re.sub(r"\?", " \? ", string)
  string = re.sub(r"\s{2,}", " ", string)
  return word_tokenize(string.strip().lower())

cleaned = [el for arr in list(map(lambda w: clean_str(w), alpha_only)) for el in arr ]
cleaned = list(filter(lambda w: len(w) > 2 and not w.replace('"', '').replace("'", '').replace(".", '').replace(",", '').isnumeric(), cleaned))
print(len(cleaned))

407229
407229


In [92]:
cleaned[:10]

['fanatiques',
 'redgravec',
 'rience',
 'buseno',
 'urgenthigh',
 'croustillants',
 'breukelen',
 'froidhillary',
 'noircies',
 'tokihiro']

In [93]:
db_corpus.allArticles.insert_one(
    {
        "id": "special_chars2",
        "corpus": cleaned,
        "description": "`only_alpha` without without special chars. WITH UPDATED ACCENT"
    }
)

### French stemming

In [48]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

print(len(alpha_only))

french_stemmed = list(map(lambda w: stemmer.stem(w), alpha_only))
french_stemmed = list(set(french_stemmed))

print(len(french_stemmed))

390040
305889


In [49]:
db_corpus.allArticles.insert_one(
    {
        "id": "alpha_french_stemmed",
        "corpus": french_stemmed,
        "description": "`greater_than_two` without numeric tokens and french\
                        stemmed"
    }
)

### English stemming

In [50]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

print(len(alpha_only))

english_stemmed = list(map(lambda w: stemmer.stem(w), alpha_only))
english_stemmed = list(set(english_stemmed))

print(len(english_stemmed))

390040
335813


In [51]:
db_corpus.allArticles.insert_one(
    {
        "id": "alpha_english_stemmed",
        "corpus": english_stemmed,
        "description": "`greater_than_two` without numeric tokens and english\
                        stemmed"
    }
)

### French and English stemmed

In [53]:
print(len(french_stemmed))

french_english_stemmed = list(map(lambda w: stemmer.stem(w), french_stemmed))
french_english_stemmed = list(set(french_english_stemmed))

print(len(french_english_stemmed))

305889
292923


In [54]:
db_corpus.allArticles.insert_one(
    {
        "id": "alpha_english_french_stemmed",
        "corpus": french_english_stemmed,
        "description": "`greater_than_two` without numeric tokens and french\
                        and english stemmed"
    }
)

## Add clean words field in all articles

In [1]:
!pip install pymongo

from pymongo import MongoClient
import numpy as np

client = MongoClient(MD)
db = client.news

    100% |████████████████████████████████| 389kB 5.1MB/s 


In [0]:
db_corpus = client.corpus

In [0]:
corpus = db_corpus.allArticles.find_one({"id": "alpha_english_french_stemmed"})["corpus"]

In [10]:
print(corpus[:10])

[u'foufoun', u'ledirectoir', u'troproug', u'gavap', u'woodi', u'gah', u'daiich', u'juvent', u'woodr', u'dwaar']


This is an attempt to update the corpus. Will be finaly done elsewhere for performane concerns.

In [0]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
stemmerFR = SnowballStemmer("french")
stemmerEN = SnowballStemmer("english")
import re
import threading

alphaOnly = lambda w: not w.replace('"', '').replace("'", '')\
          .replace(".", '').replace(",", '').isnumeric()

gt2 = lambda w: len(w) > 2

ppFilters = lambda w: alphaOnly(w) and gt2(w)

def clean_str(string):
  """Original taken
  from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
  """
  string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
  string = re.sub(r"\'s", " \'s", string)
  string = re.sub(r"\'ve", " \'ve", string)
  string = re.sub(r"n\'t", " n\'t", string)
  string = re.sub(r"\'re", " \'re", string)
  string = re.sub(r"\'d", " \'d", string)
  string = re.sub(r"\'ll", " \'ll", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", " \( ", string)
  string = re.sub(r"\)", " \) ", string)
  string = re.sub(r"\?", " \? ", string)
  string = re.sub(r"\s{2,}", " ", string)
  return string.strip().lower()

stems = lambda w: stemmerFR.stem(w) and stemmerEN.stem(w)

def ppMaps(w):
  #w = clean_str(w)
  w = stems(w)
  return w

def preprocessing(tokens):
  tokens = list(filter(ppFilters, map(ppMaps, tokens)))
  return tokens

def sec_process():
  print("Start...")
  i = 0
  #db_corpus.allArticles.find_one_and_update( {},{'$set':{"raw_more_than_two_chars": larger}} )
  for article in db.allArticles.find():
    i = i+1
    db.allArticles.update_one({'_id':article["_id"]}, {"$set": {"cleanContent": " ".join(preprocessing(word_tokenize(clean_str(article['clearedHTML']))))}}, upsert=False)
    if i % 1000 == 0:
      print("> {}", i)

  print("...Done!")


class ProcessDocs:
  
  def __init__(self):
    self.i = [0]
    cursors = db.allArticles.parallel_scan(4)
    threads = [threading.Thread(target=self.process_cursor, args=(cursor,self.i)) for cursor in cursors]

    for thread in threads:
      thread.start()

    for thread in threads:
      thread.join()

  def process_document(self, document, i):
    i[0] = i[0]+1
    db.allArticles.update_one({'_id':document["_id"]}, {"$set": {"cleanContent": " ".join(preprocessing(word_tokenize(clean_str(document['clearedHTML']))))}}, upsert=False)
    if self.i[0] % 1000 == 0:
      print("> %d", i)

  def process_cursor(self, cursor, i):
    for document in cursor:
      # Some thread-safe processing function:
      self.process_document(document, i)
      
ProcessDocs()

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
('> %d', [1000])
('> %d', [2000])
('> %d', [3000])


## Tokenize and clean for vectorizer TF-IDF ©

In [2]:
import sys
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
stemmerFR = SnowballStemmer("french")
stemmerEN = SnowballStemmer("english")
import re
from scipy import optimize

alphaOnly = lambda w: not w.replace('"', '').replace("'", '')\
          .replace(".", '').replace(",", '').isnumeric()

gt2 = lambda w: len(w) > 2

ppFilters = lambda w: alphaOnly(w) and gt2(w)

def clean_str(string):
  """Original taken
  from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
  """
  string = re.sub(r"[^A-Za-z0-9()èéàÉÀêÊçÇ]", " ", string)
  string = re.sub(r"\'s", " \'s", string)
  string = re.sub(r"\'ve", " \'ve", string)
  string = re.sub(r"n\'t", " n\'t", string)
  string = re.sub(r"\'re", " \'re", string)
  string = re.sub(r"\'d", " \'d", string)
  string = re.sub(r"\'ll", " \'ll", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", " \( ", string)
  string = re.sub(r"\)", " \) ", string)
  string = re.sub(r"\?", " \? ", string)
  string = re.sub(r"\s{2,}", " ", string)
  return string.strip().lower()


def ppMaps(w):
  #w = clean_str(w)
  w = stemmerFR.stem(w)
  #w = stemmerEN.stem(w)
  return w

def preprocessing(tokens):
  tokens = list(filter(ppFilters, map(ppMaps, tokens)))
  return tokens

def process_line(line):
  return preprocessing(word_tokenize(clean_str(line)))

# Call `pro£cess_line` to use it.

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
process_line("Hello loïc. Are you there?")

['hello', 'are', 'you', 'ther']

## Create Stemmed <> Original pairs based on alpha_only

In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

In [0]:
db_corpus = client.corpus
corpus = db_corpus.allArticles.find_one({"id": "special_chars2"})["corpus"]

In [96]:
print(corpus[:10])

['fanatiques', 'redgravec', 'rience', 'buseno', 'urgenthigh', 'croustillants', 'breukelen', 'froidhillary', 'noircies', 'tokihiro']


In [0]:
pairs = {}
for w in corpus:
  pairs[w] = stemmer.stem(w)

In [98]:
db_corpus.allArticles.insert_one(
    {
        "id": "pairs2",
        "pairs": pairs,
        "description": "each original word from special_chars` with their corresponding french stemming WITH UPDATED ACCENT"
    }
)

In [0]:
pairs_got = db_corpus.allArticles.find_one({"id": "pairs"})["pairs"]

In [100]:
print(list(pairs_got.keys())[:10])
print(list(pairs_got.values())[:10])
print(len(list(pairs_got.values())))

['poliscanova', 'applepartage', 'sowell', 'administreront', 'woods', 'sunde', 'bolognelundi', 'unescosouligne', 'woody', 'hennings']
['poliscanov', 'applepartag', 'sowel', 'administr', 'wood', 'sund', 'bolognelund', 'unescosoulign', 'woody', 'henning']
360447
